In [1]:
import json, re, sys
import pandas as pd
import numpy as np

#from Bio.KEGG import REST


In [4]:
### Generate the genome CSV

genome_file = "raw_data/bacteroidetes_2022_05_16.json"

cellulose_degrader_file = "raw_data/cellulose_info.tsv"

is_header = True

genome_degrader = {}

output_folder = "data_for_neo4j_bacteroidetes"

for line in open(cellulose_degrader_file, 'r'):
    if is_header == True:
        is_header = False
    else:
        fields = line.strip().split("\t")

        genome_degrader[fields[0]] = fields[1]

desired_rank = ["superkingdom", "phylum", "class", "order", "family", "genus", "species"]

taxon_dict = {}

parent_son = set()
taxon_genome = set()

with open(f"{output_folder}/taxon.csv", "w+") as output:
    output.write(",".join(["name", "taxid", "rank"]) + "\n")

with open(f"{output_folder}/genome.csv", "w+") as output:
    output.write(",".join(["name", "cellulose"]) + "\n")

with open(f"{output_folder}/taxon_connections.csv", "w+") as output:
    output.write(",".join(["from", "to"]) + "\n")

with open(f"{output_folder}/genome_connections.csv", "w+") as output:
    output.write(",".join(["from", "to"]) + "\n")

with open(f"{output_folder}/has_cazy.csv", "w+") as output:
    output.write(",".join(["from", "to", "amount"]) + "\n")

for line in open(genome_file, 'r'):
    genome = json.loads(line)

    if "superkingdom" in genome["taxonomy"] and genome["taxonomy"]["superkingdom"][0] in ["Bacteria", "Archaea"]:
        taxid = genome["taxonomy"]["taxid"]
        name = genome["name"]

        taxonomy = genome["taxonomy"]

        #taxonomy["genome"] = [name, genome["taxid"]]

        quartett = [""] * 2
        for rank in desired_rank:
            if rank in taxonomy:
                taxon, taxon_taxid = taxonomy[rank]

                if taxon not in taxon_dict:
                    taxon_dict[taxon] = [taxon_taxid, rank]

                if rank == "superkingdom":
                    quartett[0] = taxon
                else:
                    quartett[1] = taxon

                    #if quartett[0] != quartett[1]:
                    
                    parent_son.add(tuple(quartett))

                    quartett[0] = quartett[1]
        
        taxon_dict[name] = [taxid, "genome"]
        taxon_genome.add(tuple([quartett[0], name]))

        for cazy in genome["cazy"]:
            with open(f"{output_folder}/has_cazy.csv", "a") as output:            
                output.write(",".join([f'"{name}"', f'"{cazy}"', f'{genome["cazy"][cazy]}']) + "\n")




for name in taxon_dict:
    with open(f"{output_folder}/taxon.csv", "a") as output:
        if taxon_dict[name][1] != "genome":
            output.write(",".join([f'"{name}"', f'"{taxon_dict[name][0]}"', f'"{taxon_dict[name][1]}"']) + "\n")

for name in taxon_dict:
    with open(f"{output_folder}/genome.csv", "a") as output:
        if taxon_dict[name][1] == "genome":
            cellulose = ""

            if name in genome_degrader:
                if genome_degrader[name] == "no cellulose degrader":
                    output.write(",".join([f'"{name}"', '0']) + "\n")
                else:
                    output.write(",".join([f'"{name}"', '1']) + "\n")
            else:
                output.write(",".join([f'"{name}"', '2']) + "\n")

for parent in parent_son:
    with open(f"{output_folder}/taxon_connections.csv", "a") as output:
        #["~id", "~from", "~to", "~label"]
        output.write(",".join([f'"{parent[0]}"', f'"{parent[1]}"']) + "\n")

for parent in taxon_genome:
    with open(f"{output_folder}/genome_connections.csv", "a") as output:
        output.write(",".join([f'"{parent[0]}"', f'"{parent[1]}"']) + "\n")

In [5]:
## generate the cazy CSVs

info_file = "raw_data/cazy_info_2022_05_16.json"

output_folder = "data_for_neo4j_bacteroidetes"
rx_ec = re.compile(r'\b\d+\.\d+\.\d+\.\d+\b')

with open(f"{output_folder}/cazy.csv", "w") as output:
    output.write(",".join(["name", "activities"]) + "\n")

# with open("ec.csv", "w") as output:
#     output.write(",".join(["name", "sysname", "reaction"]) + "\n")

# with open("cazy_ec.csv", "w") as output:
#     output.write(",".join(["from", "to"]) + "\n")

#unique_ec = set()

for line in open(info_file, 'r'):
    cazy_line = json.loads(line)

    cazy = cazy_line["name"]
    activities = ""


    if "activity" in cazy_line:
        activities = re.sub(r'<.+?>', '', cazy_line["activity"])


    # if "ec" in cazy_line:
    #     ecs = cazy_line["ec"]

    #     for ec in ecs:
    #         match_ec = rx_ec.match(ec)

    #         if match_ec:
    #             with open(f"{output_folder}cazy_ec.csv", "a") as output:
    #                 output.write(",".join([f'"{cazy}"', f'"{ec}"']) + "\n")
                
    #             unique_ec.add(ec)
        
    with open(f"{output_folder}/cazy.csv", "a") as output:
        output.write(",".join([f'"{cazy}"', f'"{activities}"']) + "\n")

# for ec in unique_ec:
#     with open(f"{output_folder}ec.csv", "a") as output:
#         output.write(",".join([f'"{ec}"']) + "\n")

In [21]:
### generate the EC CSVs

with open("ec_final.csv", "w") as output:
    output.write(",".join(["name", "sysname", "reaction"]) + "\n")

is_header = True
for line in open("ec.csv"):

    if is_header == True:
        is_header = False
    else:

        ec = f"ec:{line.strip()}"
        sysname = ""
        reaction = ""
        for content in REST.kegg_get(ec):
            for l in content.split("\n"):
                if l.startswith("SYSNAME"):
                    sysname = l.replace("SYSNAME", "").strip()
            
                if l.startswith("REACTION"):
                    reaction = l.replace("REACTION", "").strip()
        
        #print (f'{line.strip()},"{sysname}","{reaction}"')
        with open("ec_final.csv", "a") as output:
            output.write(",".join([line.strip(), f'"{sysname}"', f'"{reaction}"']) + "\n")